In [3]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from Classes.CartTreeClassifier import CartTreeClassifier
from Classes.CartTreeTeacher import CartTreeTeacher

from Functions.metrics import *
from Functions.split_functions import *
from Functions.spreading_functions import *
from Functions.search_split_functions import *

from Functions.metrics import accuracy
from sys import setrecursionlimit
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler

from Classes.LayerGen import LayerGenerator
from Classes.Layer import Layer
from Classes.Perceptron import Perceptron
from Classes.PerceptronBWSearcher import PerceptronBWSearcher


In [8]:
data = pd.read_csv('../Data/DataLaba2/neo_task_smoted.csv')
mm_scaler = MinMaxScaler()
data_scaled = mm_scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)
data_scaled_x, hazardous = SMOTE().fit_resample(data_scaled.drop(['hazardous'], axis=1), data['hazardous'])
data_scaled_x = pd.DataFrame(data_scaled_x, columns=data.columns[data.columns != 'hazardous'])
X_train, X_test, y_train, y_test = train_test_split(data_scaled, hazardous, test_size=0.5, random_state=42)
